# Face Recognition

#### Load Packages

In [15]:
# Install OpenCV
!pip install opencv-python

# Install TensorFlow
!pip install tensorflow

# Install Keras
!pip install keras

# Install pandas
!pip install pandas

# Install numpy
!pip install numpy

# Ensure matplotlib is installed
!pip install matplotlib


  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.12.23-py2.py3-none-any.whl.metadata (876 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.69.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached ml_dtypes-0.4.1-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\mjawa\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\mjawa\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\mjawa\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\mjawa\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mjawa\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\mjawa\anaconda3\Lib\socket.py", line 708, in readinto
    return self._sock.recv_into(b)
           ^


   --------- ----------------------------- 98.9/390.3 MB 105.0 kB/s eta 0:46:16
   --------- ----------------------------- 98.9/390.3 MB 105.0 kB/s eta 0:46:16
   --------- ----------------------------- 98.9/390.3 MB 105.0 kB/s eta 0:46:17
   --------- ----------------------------- 98.9/390.3 MB 105.0 kB/s eta 0:46:17
   --------- ----------------------------- 98.9/390.3 MB 105.0 kB/s eta 0:46:17
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- ----------------------------- 98.9/390.3 MB 105.3 kB/s eta 0:46:08
   --------- --------------------------

In [16]:
# Import necessary libraries

import os
import numpy as np
import pandas as pd
import cv2  # For image processing
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import (
    Conv2D, ZeroPadding2D, Activation, Input, BatchNormalization, MaxPooling2D,
    AveragePooling2D, Concatenate, Lambda, Flatten, Dense
)
from keras.initializers import glorot_uniform  # Xavier initialization
from keras import backend as K
from fr_utils import *  # Custom utility functions
from inception_blocks_v2 import *  # Custom inception block implementation

# Jupyter-specific configuration
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Suppress scientific notation for large arrays
np.set_printoptions(suppress=True)

# Set TensorFlow's default image data format (Modern practice uses channels_last by default)
K.set_image_data_format('channels_first')  # If required by your pre-trained models


ModuleNotFoundError: No module named 'tensorflow'

## 0 - Naive Face Verification

## 1 - Encoding face images in 128-dimensional vector

### 1.1 - Using a ConvNet to compute encodings

In [ ]:
# Initialize the face recognition model with the input shape
FRmodel = faceRecoModel(input_shape=(3, 96, 96))


In [ ]:
# Print the total number of parameters in the model
print("Total Params:", FRmodel.count_params())


### 1.2 The Triplet Loss

In [ ]:
import tensorflow as tf

# Define the triplet loss function
def triplet_loss(y_true, y_pred, alpha=0.2):
    """
    Computes the triplet loss as per the formula:
    L = max(||f(a) - f(p)||^2 - ||f(a) - f(n)||^2 + alpha, 0)
    
    Arguments:
    y_true -- True labels (not used, required by Keras loss function signature).
    y_pred -- A list containing three tensors:
              anchor    - Encodings for anchor images, shape (batch_size, embedding_size).
              positive  - Encodings for positive images, shape (batch_size, embedding_size).
              negative  - Encodings for negative images, shape (batch_size, embedding_size).
    alpha -- Margin parameter for the triplet loss (default is 0.2).
    
    Returns:
    loss -- Scalar, the computed triplet loss value.
    """
    # Extract anchor, positive, and negative encodings from predictions
    anchor, positive, negative = y_pred

    # Compute the squared distances between anchor-positive and anchor-negative
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)  # ||f(a) - f(p)||^2
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)  # ||f(a) - f(n)||^2

    # Compute the basic loss: (positive distance - negative distance + alpha)
    basic_loss = pos_dist - neg_dist + alpha

    # Apply the ReLU activation to ensure the loss is non-negative
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))  # max(basic_loss, 0)

    return loss


In [ ]:
# Set the random seed for reproducibility
tf.random.set_seed(1)

# Define random tensors for the anchor, positive, and negative encodings
y_pred = (
    tf.random.normal([3, 128], mean=6, stddev=0.1, seed=1),
    tf.random.normal([3, 128], mean=1, stddev=1, seed=1),
    tf.random.normal([3, 128], mean=3, stddev=4, seed=1),
)

# Compute the triplet loss
loss = triplet_loss(None, y_pred)

# Print the computed loss value
print(f"Loss = {loss.numpy():.4f}")


## 2 - Loading the pre-trained model

In [ ]:
# Compile the Face Recognition model
# Using Adam optimizer and triplet loss as the loss function
FRmodel.compile(
    optimizer='adam', 
    loss=triplet_loss, 
    metrics=['accuracy']  # Note: Accuracy may not be meaningful for triplet loss
)

# Load pre-trained weights into the model
load_weights_from_FaceNet(FRmodel)


## 3 - Applying the model

### 3.1 - Face Verification

In [ ]:
# Initialize an empty database for storing encodings
database = {}

# List of user names and corresponding image file paths
user_images = {
    "danielle": "images/danielle.png",
    "younes": "images/younes.jpg",
    "tian": "images/tian.jpg",
    "andrew": "images/andrew.jpg",
    "kian": "images/kian.jpg",
    "dan": "images/dan.jpg",
    "sebastiano": "images/sebastiano.jpg",
    "bertrand": "images/bertrand.jpg",
    "kevin": "images/kevin.jpg",
    "felix": "images/felix.jpg",
    "benoit": "images/benoit.jpg",
    "arnaud": "images/arnaud.jpg",
}

# Populate the database with image encodings
for name, image_path in user_images.items():
    try:
        # Encode the image using the FRmodel
        database[name] = img_to_encoding(image_path, FRmodel)
    except FileNotFoundError:
        print(f"Error: Image file '{image_path}' not found.")
    except Exception as e:
        print(f"Error encoding image for {name}: {e}")


In [ ]:
def verify(image_path: str, identity: str, database: dict, model) -> tuple:
    """
    Verifies if the person in the provided image matches the specified identity.

    Args:
    - image_path (str): Path to the input image.
    - identity (str): Name of the person to verify. Must exist in the database.
    - database (dict): Mapping of names to their respective encodings (vectors).
    - model: Inception model instance for encoding.

    Returns:
    - dist (float): The computed distance between the input image encoding and the stored encoding for the identity.
    - door_open (bool): True if the door should open, False otherwise.
    """
    try:
        # Step 1: Compute the encoding for the input image
        encoding = img_to_encoding(image_path, model)

        # Step 2: Calculate the Euclidean distance between the input encoding and the database encoding for the identity
        dist = np.linalg.norm(encoding - database[identity])

        # Step 3: Determine if the door should open based on the distance threshold
        if dist < 0.7:
            print(f"It's {identity}, welcome in!")
            door_open = True
        else:
            print(f"It's not {identity}, please go away.")
            door_open = False

    except KeyError:
        # Handle case where the identity is not in the database
        print(f"Error: Identity '{identity}' not found in the database.")
        return None, False
    except FileNotFoundError:
        # Handle missing image file
        print(f"Error: Image file '{image_path}' not found.")
        return None, False
    except Exception as e:
        # Catch all other exceptions
        print(f"An unexpected error occurred: {e}")
        return None, False

    return dist, door_open


In [ ]:
# Verify if "camera_0.jpg" matches "younes" in the database
verify("images/camera_0.jpg", "younes", database, FRmodel)


In [ ]:
# Verify if "kian" matches the face in "images/camera_2.jpg" using the FRmodel and database.
verify("images/camera_2.jpg", "kian", database, FRmodel)


### 3.2 - Face Recognition

In [ ]:
def who_is_it(image_path, database, model):
    """
    Identifies the person in the given image by finding the closest match in the database.
    
    Arguments:
    image_path -- str: Path to the image for identification.
    database -- dict: A dictionary with names as keys and image encodings as values.
    model -- Keras model: Pre-trained Inception model for generating image encodings.
    
    Returns:
    min_dist -- float: Minimum distance between the input image encoding and the database encodings.
    identity -- str: Name of the identified person from the database, or None if not found.
    """
    # Compute the encoding for the input image.
    encoding = img_to_encoding(image_path, model)

    # Initialize minimum distance to a high value and identity to None.
    min_dist = float('inf')  # Use infinity for clarity in comparisons.
    identity = None

    # Iterate through the database to find the closest match.
    for name, db_enc in database.items():
        # Compute the L2 (Euclidean) distance between encodings.
        dist = np.linalg.norm(encoding - db_enc)

        # Update the minimum distance and identity if a closer match is found.
        if dist < min_dist:
            min_dist = dist
            identity = name

    # Print the result based on the threshold value.
    if min_dist > 0.7:
        print("Not in the database.")  # No match found.
    else:
        print(f"It's {identity}, the distance is {min_dist:.2f}")  # Match found.

    return min_dist, identity


In [ ]:
# Identify the person in the given image using the face recognition model.
who_is_it("images/camera_0.jpg", database, FRmodel)


### Future work
#### Ways to improve your facial recognition model
Although we won't implement it here, here are some ways to further improve the algorithm:
- Put more images of each person (under different lighting conditions, taken on different days, etc.) into the database. Then given a new image, compare the new face to multiple pictures of the person. This would increase accuracy.
- Crop the images to just contain the face, and less of the "border" region around the face. This preprocessing removes some of the irrelevant pixels around the face, and also makes the algorithm more robust.


### References:

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- The pretrained model we use is inspired by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.
- Our implementation also took a lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet 
